In [1]:
import tensorflow as tf
from d2l import tensorflow as d2l

In [2]:
# @save
def masked_softmax(X, valid_lens):
    """通过在最后一个轴上掩蔽元素来执行softmax操作"""
    # X:3D张量，valid_lens:1D或2D张量
    if valid_lens is None:
        return tf.nn.softmax(X, axis=-1)
    else:
        shape = X.shape
        if len(valid_lens.shape) == 1:
            valid_lens = tf.repeat(valid_lens, repeats=shape[1])

        else:
            valid_lens = tf.reshape(valid_lens, shape=-1)
        # 最后一轴上被掩蔽的元素使用一个非常大的负值替换，从而其softmax输出为0
        X = d2l.sequence_mask(
            tf.reshape(X, shape=(-1, shape[-1])), valid_lens, value=-1e6
        )
        return tf.nn.softmax(tf.reshape(X, shape=shape), axis=-1)


In [3]:
masked_softmax(tf.random.uniform(shape=(2, 2, 4)), tf.constant([2, 3]))


<tf.Tensor: shape=(2, 2, 4), dtype=float32, numpy=
array([[[0.33927715, 0.6607229 , 0.        , 0.        ],
        [0.5186399 , 0.48136008, 0.        , 0.        ]],

       [[0.2706828 , 0.34843233, 0.38088483, 0.        ],
        [0.28000477, 0.29489115, 0.42510405, 0.        ]]], dtype=float32)>

In [4]:
masked_softmax(tf.random.uniform(shape=(2, 2, 4)), tf.constant([[1, 3], [2, 4]]))


<tf.Tensor: shape=(2, 2, 4), dtype=float32, numpy=
array([[[1.        , 0.        , 0.        , 0.        ],
        [0.30077505, 0.25723618, 0.4419887 , 0.        ]],

       [[0.5924722 , 0.40752774, 0.        , 0.        ],
        [0.19095424, 0.21986324, 0.22264503, 0.36653754]]], dtype=float32)>

In [5]:
# @save
class AdditiveAttention(tf.keras.layers.Layer):
    """Additiveattention."""

    def __init__(self, key_size, query_size, num_hiddens, dropout, **kwargs):
        super().__init__(**kwargs)
        self.W_k = tf.keras.layers.Dense(num_hiddens, use_bias=False)
        self.W_q = tf.keras.layers.Dense(num_hiddens, use_bias=False)
        self.w_v = tf.keras.layers.Dense(1, use_bias=False)
        self.dropout = tf.keras.layers.Dropout(dropout)

    def call(self, queries, keys, values, valid_lens, **kwargs):
        queries, keys = self.W_q(queries), self.W_k(keys)
        # 在维度扩展后，
        # queries的形状：(batch_size，查询的个数，1，num_hidden)
        # key的形状：(batch_size，1，“键－值”对的个数，num_hiddens)
        # 使用广播方式进行求和
        features = tf.expand_dims(queries, axis=2) + tf.expand_dims(keys, axis=1)
        features = tf.nn.tanh(features)
        # self.w_v仅有一个输出，因此从形状中移除最后那个维度。
        # scores的形状：(batch_size，查询的个数，“键-值”对的个数)
        scores = tf.squeeze(self.w_v(features), axis=-1)
        self.attention_weights = masked_softmax(scores, valid_lens)
        # values的形状：(batch_size，“键－值”对的个数，值的维度)
        return tf.matmul(self.dropout(self.attention_weights, **kwargs), values)


In [6]:
# @save
class DotProductAttention(tf.keras.layers.Layer):
    """Scaleddotproductattention."""

    def __init__(self, dropout, **kwargs):
        super().__init__(**kwargs)
        self.dropout = tf.keras.layers.Dropout(dropout)

    # queries的形状：(batch_size，查询的个数，d)
    # keys的形状：(batch_size，“键－值”对的个数，d)
    # values的形状：(batch_size，“键－值”对的个数，值的维度)
    # valid_lens的形状:(batch_size，)或者(batch_size，查询的个数)
    def call(self, queries, keys, values, valid_lens, **kwargs):
        d = queries.shape[-1]
        scores = tf.matmul(queries, keys, transpose_b=True) / tf.math.sqrt(
            tf.cast(d, dtype=tf.float32)
        )
        self.attention_weights = masked_softmax(scores, valid_lens)
        return tf.matmul(self.dropout(self.attention_weights, **kwargs), values)
